# Web Scraping - Wikipedia

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from ClassScrapingWiki import * #Importing python file containing class for Web Scraping

In [2]:
#Top US cities by population Wikipedia Link
link='https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'

In [3]:
#Extracting Wikipedia Table
results = requests.get(link) #HTTP Request
city_soup=BeautifulSoup(results.text,'lxml')
scrap=Scraping() #Creating Instance for the Scraping Class
table_link_ext=city_soup.find('table',{'class':'wikitable sortable'})
table=scrap.Extract_Table(table_link_ext) #Function call to extract the Wikipedia Table
table.head(3) #Displaying Head of the Extracted Table

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area (Sq.Km),2016 population density,population density (per Sq.Km),Location
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...


In [4]:
#Data Cleansing up the Extracted Wikipedia Table
table['City'] = table['City'].replace('(\[.*?\])','', regex=True)
table['2016 land area'] = table['2016 land area'].replace('[a-z]','', regex=True)
table['population density (per Sq.Km)']=table['population density (per Sq.Km)'].apply(lambda x: x.str.split('/').str[0],axis=1)
table['2016 population density']=table['2016 population density'].apply(lambda x: x.str.split('/').str[0],axis=1)
table['Location']=table['Location'].apply(lambda x: x.str.split('/').str[0],axis=1)
table['2016 land area (Sq.Km)']=table['2016 land area (Sq.Km)'].apply(lambda x: x.str[:-4],axis=1)
table['City'] = table['City'].replace('[^0-9a-zA-Z/ )(]+','', regex=True)
table.rename(columns={'State[c]': 'State','2016 land area':'2016 land area (Sq.mi)','2016 population density':'2016 population density (per Sq.mi)'}, inplace=True) #Renaming Column Names
table['2016 land area (Sq.mi)']=table['2016 land area (Sq.mi)'].apply(lambda x: x.str.strip())
table.head(10) #Displaying Head of the Data Cleansed Table

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area (Sq.mi),2016 land area (Sq.Km),2016 population density (per Sq.mi),population density (per Sq.Km),Location
0,1,New York City,New York,"8,398,748","8,175,133",+2.74%,301.5,780.9,"28,317","10,933",40°39′49″N 73°56′19″W﻿
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7,"1,213.9","8,484","3,276",34°01′10″N 118°24′39″W﻿
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3,588.7,"11,900","4,600",41°50′15″N 87°40′54″W﻿
3,4,Houston,Texas,"2,325,502","2,100,263",+10.72%,637.5,"1,651.1","3,613","1,395",29°47′12″N 95°23′27″W﻿
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6,"1,340.6","3,120","1,200",33°34′20″N 112°05′24″W﻿
5,6,Philadelphia,Pennsylvania,"1,584,138","1,526,006",+3.81%,134.2,347.6,"11,683","4,511",40°00′34″N 75°08′00″W﻿
6,7,San Antonio,Texas,"1,532,233","1,327,407",+15.43%,461.0,"1,194.0","3,238","1,250",29°28′21″N 98°31′30″W﻿
7,8,San Diego,California,"1,425,976","1,307,402",+9.07%,325.2,842.3,"4,325","1,670",32°48′55″N 117°08′06″W﻿
8,9,Dallas,Texas,"1,345,047","1,197,816",+12.29%,340.9,882.9,"3,866","1,493",32°47′36″N 96°45′59″W﻿
9,10,San Jose,California,"1,030,119","945,942",+8.90%,177.5,459.7,"5,777","2,231",37°17′48″N 121°49′08″W﻿


**Moving onto Extracting Individual City Pages**

In [5]:
#Extracting Links of Individual City pages
links=scrap.Extract_Links(city_soup.find('table',{'class':'wikitable sortable'}))

In [6]:
#Extracting information from individual city pages
count=0
for link in links:
    if count==0:
        Final_out=scrap.CityPageScrap(link)
        count+=1
    else:
        temp=scrap.CityPageScrap(link)
        if temp is None: continue
        Final_out=pd.concat([Final_out,temp],ignore_index=True,sort=False)

In [7]:
#Fixing Up some Issues in Web Scraping along the way
Final_out['State']=Final_out['State'].fillna(Final_out['US state'])
Final_out.loc[Final_out['CityName'] =='Washington, D.C.', 'State']='District of Columbia'
Final_out['CityName']=Final_out['CityName'].apply(lambda x: x.split(',')[0])
Final_out.loc[Final_out['State'] =='District of Columbia', 'CityName']='Washington DC'
Final_out['CityName'] = Final_out['CityName'].replace('[^0-9a-zA-Z/ )(]+','', regex=True)
Final_out['CityName']=Final_out['CityName'].str.replace('Township','')
Final_out['CityName']=Final_out['CityName'].str.strip()
Final_out.head(3)

In [8]:
#Filtering out columns containing less than 10% of values
Final_out=Final_out[Final_out.columns[Final_out.isnull().mean() < 0.9]] 
Final_out.head(3)

,Country,State,Settled,Named for,Type,Body,Mayor,Total,Land,Water,...,Commuter Rail,Interstates,ZIP codes,State Routes,Founded,ZIP code,City Manager,City Council,Primary Airport,City manager
0,United States,New York,1624,"James, Duke of York",Mayor–Council,New York City Council,Bill de Blasio (D),"468.484 sq mi (1,213.37 km2)",302.643 sq mi (783.84 km2),165.841 sq mi (429.53 km2),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,California,NaN,"Our Lady, Queen of the Angels",Mayor-Council-Commission,Los Angeles City Council,Eric Garcetti (D),"502.76 sq mi (1,302.15 km2)","468.74 sq mi (1,214.03 km2)",34.02 sq mi (88.12 km2) 6.7%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,Illinois,circa 1780,Miami-Illinois: shikaakwa(wild onion or wild g...,Mayor–council,Chicago City Council,Lori Lightfoot (D),NaN,227.34 sq mi (588 km2),6.80 sq mi (17.62 km2) 3.0%,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Merging and Removing the Redundant columns
Final_out['ZIP Codes']=Final_out['ZIP Codes'].fillna(Final_out['ZIP codes'])
Final_out['ZIP Codes']=Final_out['ZIP Codes'].fillna(Final_out['ZIP code'])
Final_out['Area codes']=Final_out['Area codes'].fillna(Final_out['Area code'])
Final_out['City Manager']=Final_out['City Manager'].fillna(Final_out['City manager'])
Final_out['Counties']=Final_out['Counties'].fillna(Final_out['County'])
Final_out['GNIS feature IDs']=Final_out['GNIS feature IDs'].fillna(Final_out['GNIS feature ID'])
Final_out['Incorporated']=Final_out['Incorporated'].fillna(Final_out['Incorporated '])
Final_out=Final_out.drop(['ZIP codes','ZIP code','Area code','City manager','County','GNIS feature ID','Incorporated ','Rank'],axis=1) #Dropping Redundant columns

In [10]:
#Data Cleansing the Individual City Pages DataSet before Merging

Final_out['Settled']=Final_out['Settled'].replace('[^0-9/,)(]+','', regex=True)
Final_out['Settled']=Final_out['Settled'].apply(lambda x: x.split(',')[1] if str(x).find(',')!=-1 else x)

Final_out['Total']=Final_out['Total'].apply(lambda x: str(x).split('(')[0])
Final_out['Land']=Final_out['Land'].apply(lambda x: str(x).split('(')[0])
Final_out['Water']=Final_out['Water'].apply(lambda x: str(x).split('(')[0])
Final_out['Metro']=Final_out['Metro'].apply(lambda x: str(x).split('(')[0])
Final_out['Elevation']=Final_out['Elevation'].apply(lambda x: str(x).split('(')[0])

Final_out['Total']=Final_out['Total'].replace('nan',np.nan)
Final_out['Land']=Final_out['Land'].replace('nan',np.nan)
Final_out['Water']=Final_out['Water'].replace('nan',np.nan)
Final_out['Metro']=Final_out['Metro'].replace('nan',np.nan)
Final_out['Elevation']=Final_out['Elevation'].replace('nan',np.nan)

Final_out['Total']=Final_out['Total'].replace('[^0-9.]+','', regex=True)
Final_out['Land']=Final_out['Land'].replace('[^0-9.]+','', regex=True)
Final_out['Water']=Final_out['Water'].replace('[^0-9.]+','', regex=True)
Final_out['Metro']=Final_out['Metro'].replace('[^0-9,.\u2014]+','', regex=True)

Final_out['Elevation']=Final_out['Elevation'].replace('[^0-9.,-\u2014]+','', regex=True)

Final_out['Estimate']=Final_out['Estimate'].replace('[^0-9.,]+','', regex=True)

Final_out['Density']=Final_out['Density'].apply(lambda x: str(x).split(' ')[0])
Final_out['Density']=Final_out['Density'].replace('nan',np.nan)
Final_out['Density']=Final_out['Density'].replace('[^0-9.,]+','', regex=True)

Final_out['Urban']=Final_out['Urban'].replace('\([^)]*\)','', regex=True)
Final_out['Urban']=Final_out['Urban'].replace('[^0-9.,]+','', regex=True)

Final_out['City']=Final_out['City'].replace('\([^)]*\)','', regex=True)
Final_out['City']=Final_out['City'].replace('[^0-9.,]+','', regex=True)

In [11]:
#Renaming Columns
Final_out.rename(columns={'Total': 'Total (sq.mi)',\
                          'Land':'Land (sq.mi)',\
                          'Water':'Water (sq.mi)',\
                          'Elevation':'Elevation (in ft)',\
                          'Density':'Density (per sq.mi)',\
                          'Urban':'Urban (sq.mi)',\
                          'City':'City Area (sq.mi)'}, inplace=True)

In [12]:
#Removing MultiIndex and Merging the Datasets Together
table.columns=table.columns.get_level_values(0)
Wiki_Out=table.merge(Final_out,left_on=['City','State'],right_on=['CityName','State'])
Wiki_Out=Wiki_Out.drop(['CityName'],axis=1)
Wiki_Out.head(3) #Displaying Head of the Final Cleansed Table

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area (Sq.mi),2016 land area (Sq.Km),2016 population density (per Sq.mi),population density (per Sq.Km),...,GNIS feature IDs,Counties,City Area (sq.mi),Commuter Rail,Interstates,State Routes,Founded,City Manager,City Council,Primary Airport
0,1,New York City,New York,"8,398,748","8,175,133",+2.74%,301.5,780.9,"28,317","10,933",...,975772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7,"1,213.9","8,484","3,276",...,"1662328, 2410877",Los Angeles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3,588.7,"11,900","4,600",...,0428803,"Cook, DuPage",234.14,,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Exporting the Final Data for to be uploaded to Google Big Query
Wiki_Out.to_csv('WikiWebScrapingFinal.csv',index = False)